In [12]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import pandas as pd

In [ ]:
'UBM', 'PRISM', 'MODIS16', 'SNODAS'

In [111]:
data = "E:/PROJECTS/Round_Valley/Data/fixedbdgt.csv"
rvdataraw = pd.read_csv(data,memory_map=True)
rvdataraw = rvdataraw[rvdataraw['SOURCE'].isin(['UBM','SNODAS'])]
rvdataraw['variable'].unique()

array(['runoff', 'recharge', 'soil water', 'evapotranspiration',
       'snowmelt', 'precip as rain'], dtype=object)

In [115]:
def summarizedata(rvdataraw, varb):

    rvdata = rvdataraw[rvdataraw['HUC_12'].isin([160202030404,160202030403,160202030402])]
    rvdata = rvdata[rvdata['variable']==varb]
    rvdata['date'] = rvdata['YearMonth'].apply(lambda x: pd.to_datetime(x,format='%Y%m'),1)
    #rvdata['year'] = rvdata['YearMonth'].apply(lambda x: pd.to_datetime(x,format='%Y%m').year,1)
    #rvdata['month'] = rvdata['YearMonth'].apply(lambda x: pd.to_datetime(x, format='%Y%m').month,1)
    rvdata.set_index(['date'],inplace=True)
    month_mean = rvdata.groupby([rvdata.index.month,rvdata['HUC_12']]).mean()
    year_mean = month_mean.groupby(month_mean.index.get_level_values(1)).sum()
    by_month = rvdata.groupby(pd.TimeGrouper('MS')).sum()
    by_year = rvdata.groupby(rvdata.index.year).sum()
    monthly_mean = month_mean.groupby(month_mean.index.get_level_values(0)).sum()
    return month_mean, year_mean, monthly_mean, by_month,by_year

In [116]:
component={}
for compon in rvdataraw['variable'].unique():
    component[compon] = summarizedata(rvdataraw, compon)
    print('{:} = {:}'.format(compon,component[compon][1].volume_acft.sum()))

runoff = 1671.973442851648
recharge = 6694.92819972026
soil water = 463778.73842820164
evapotranspiration = 74056.87420323209
snowmelt = 28589.24964059479
precip as rain = 41598.464052473544


In [82]:
def getsum(comp):
    return component[comp][1].volume_acft.sum()

# Yearly UBM

In [ ]:
data = "E:/PROJECTS/Round_Valley/Data/fixedbdgt.csv"
rvdataraw = pd.read_csv(data,memory_map=True)
rvdataraw = rvdataraw[rvdataraw['SOURCE'].isin(['UBM','SNODAS'])]
rvdataraw['variable'].unique()

component={}
for compon in rvdataraw['variable'].unique():
    component[compon] = summarizedata(rvdataraw, compon)
    print('{:} = {:}'.format(compon,component[compon][1].volume_acft.sum()))

In [121]:
yearly_ubm = pd.concat([component['evapotranspiration'][4].volume_acft,
           component['precip as rain'][4].volume_acft,
           component['snowmelt'][4].volume_acft,
           component['runoff'][4].volume_acft,
           component['recharge'][4].volume_acft,
           component['soil water'][4].volume_acft,
          ],axis=1)
yearly_ubm.columns =  ['evapotranspiration','precip as rain','snowmelt', 'runoff', 'recharge', 'soil water'] 
yearly_ubm['total_in'] = yearly_ubm['snowmelt'] + yearly_ubm['precip as rain']
yearly_ubm['diff'] = yearly_ubm['total_in'] - yearly_ubm['evapotranspiration']

In [128]:
component['evapotranspiration'][1]

,COUNT,AREA,MIN,MAX,RANGE,MEAN,STD,SUM,YearMonth,HUC_10,HUC_08,volume_m_cubed,volume_acft
HUC_12,,,,,,,,,,,,,
160202030402,504,504000000,0.071826,1.485818,1.413992,0.490539,0.455997,20.602620,2410878,19224243648,192242436,2.060262e+07,16702.832200
160202030403,1044,1044000000,0.005709,1.734356,1.728647,0.491474,0.531332,42.758235,2410878,19224243648,192242436,4.275823e+07,34664.699475
160202030404,684,684000000,0.024309,1.726749,1.702440,0.490998,0.584302,27.986864,2410878,19224243648,192242436,2.798686e+07,22689.342529


In [127]:
yearly_ubm=yearly_ubm.iloc[2:-1,:]
yearly_ubm.describe()

,evapotranspiration,precip as rain,snowmelt,runoff,recharge,soil water,total_in,diff
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,67830.387074,42440.606078,33380.357954,2251.312144,7323.247751,463186.516877,75820.964032,7990.576958
std,13167.981217,13088.889252,13425.251177,3649.862264,7498.375220,12528.466950,13277.921430,5613.175886
min,50985.326843,33843.204773,21602.974198,42.616119,1281.515607,446328.810097,59047.412549,1691.351075
25%,61430.436902,35007.155012,23139.949143,225.990269,2740.540104,458214.974443,68616.898988,5593.778429
50%,69292.783104,37040.093710,30807.898576,634.582716,4981.081882,465343.447334,77624.367060,7478.336627
75%,75692.733277,44473.544776,41048.307387,2659.904591,9563.789529,470314.989768,84828.432103,9875.135156
max,81750.655243,61839.032120,50302.660464,7693.467026,18049.311631,475730.362745,88987.709459,15314.283504


# Yearly Modis and PRISM

In [ ]:
data = "E:/PROJECTS/Round_Valley/Data/fixedbdgt.csv"
rvdataraw = pd.read_csv(data,memory_map=True)
rvdataraw = rvdataraw[rvdataraw['SOURCE'].isin(['PRISM','MODIS16'])]
rvdataraw['variable'].unique()

component={}
for compon in rvdataraw['variable'].unique():
    component[compon] = summarizedata(rvdataraw, compon)
    print('{:} = {:}'.format(compon,component[compon][1].volume_acft.sum()))

In [110]:
yearly = pd.concat([component['evapotranspiration'][4].volume_acft,component['precipitation'][4].volume_acft],axis=1)

yearly.columns = ['ET','ppt']
yearly = yearly.iloc[5:,:]
yearly['diff'] = yearly.ppt - yearly.ET
yearly

,ET,ppt,diff
date,,,
2005-01-31,210589.852443,557484.897368,346895.044926
2006-01-31,207436.337196,539934.648923,332498.311727
2007-01-31,221184.506382,560860.946353,339676.439971
2008-01-31,241788.802703,505518.365060,263729.562357
2009-01-31,217809.747212,535897.663251,318087.916039
2010-01-31,213534.771110,529380.920780,315846.149670
2011-01-31,211658.049544,522458.225680,310800.176137
2012-01-31,210133.826047,543517.905027,333384.078980
2013-01-31,246160.739911,531551.863557,285391.123646


In [87]:
getsum('precipitation') - getsum('evapotranspiration')

46384.547505455586

In [83]:
(getsum('precip as rain') + getsum('snowmelt')) - getsum('evapotranspiration')

-152345.5451171929

In [60]:
snow = summarizedata(rvdataraw, 'snowmelt')
print(snow[1].volume_acft.sum())


evap = summarizedata('evapotranspiration')
print(evap[1].volume_acft.sum())




28589.2496406
159716.326861


152719.16800052088